In [1]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', cache_folder="C:\\Users\\aanur\\projects\\jobs-data-mining\\mount\\all-MiniLM-L6-v2")
embeddings = model.encode(sentences)
print(embeddings)

c:\Users\aanur\projects\jobs-data-mining\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\aanur\projects\jobs-data-mining\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[[ 6.76568747e-02  6.34959042e-02  4.87131104e-02  7.93049186e-02
   3.74480747e-02  2.65279226e-03  3.93749364e-02 -7.09846150e-03
   5.93614243e-02  3.15370485e-02  6.00980632e-02 -5.29051423e-02
   4.06067632e-02 -2.59308349e-02  2.98428573e-02  1.12692418e-03
   7.35149235e-02 -5.03819659e-02 -1.22386612e-01  2.37028170e-02
   2.97265369e-02  4.24768887e-02  2.56337821e-02  1.99516350e-03
  -5.69190495e-02 -2.71598101e-02 -3.29035521e-02  6.60248548e-02
   1.19007118e-01 -4.58791517e-02 -7.26214945e-02 -3.25839520e-02
   5.23413680e-02  4.50552925e-02  8.25300440e-03  3.67023833e-02
  -1.39415199e-02  6.53919056e-02 -2.64272559e-02  2.06394732e-04
  -1.36643406e-02 -3.62809859e-02 -1.95043553e-02 -2.89738458e-02
   3.94270159e-02 -8.84090960e-02  2.62424420e-03  1.36713535e-02
   4.83062714e-02 -3.11566126e-02 -1.17329188e-01 -5.11689894e-02
  -8.85287523e-02 -2.18962375e-02  1.42986150e-02  4.44167890e-02
  -1.34815453e-02  7.43392333e-02  2.66382713e-02 -1.98762380e-02
   1.79191

c:\Users\aanur\projects\jobs-data-mining\.venv\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [2]:
model.to('cuda')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [3]:
model.device

device(type='cuda', index=0)

In [9]:
import time 
start = time.time()
embeddings = model.encode(sentences[0])
en = time.time()
print(en-start)

0.03600955009460449


In [10]:
type(embeddings)

numpy.ndarray

In [12]:
embeddings.shape

(384,)

In [2]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [27]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="top_skills",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

True

In [46]:
client.collection_exists(collection_name="top_skills")

True

In [45]:
type(Distance.COSINE)

<enum 'Distance'>

In [5]:
from data_miner.db.top_skills import TopSkillsTable, DB_Creds

db_creds = DB_Creds(
    host="localhost",
    port="3306",
    user="local",
    password="local",
    database="jobs_data"
)

top_skills_table = TopSkillsTable(db_creds)

In [6]:
top_skills = top_skills_table.get_all_top_skills()

In [64]:
for top_sil in top_skills.iloc[:10].itertuples():
    print(top_sil)

Pandas(Index=0, id=1, entry_date=Timestamp('2024-09-01 06:38:10'), job_id='%E2%80%8B%E2%80%8Bsoftware-engineer%E2%80%8B-at-microsoft-3974756992', top_skills='Proficient coding skills in Java or Scala or Python#Knowledge of SQL#Familiarity with distributed systems like Spark#Git#Maven#Azure DevOps#Data Engineering#Data Science#Data Integration#Experience in Cloud Developer toolchains (AWS or MS Azure)', number_of_skills=10, exact_llm_output='Based on the job description, here are the top skills required for the role:\n\n1. Proficient coding skills in Java or Scala or Python\n2. Knowledge of SQL\n3. Familiarity with distributed systems like Spark\n4. Git\n5. Maven\n6. Azure DevOps\n7. Data Engineering\n8. Data Science\n9. Data Integration\n10. Experience in Cloud Developer toolchains (AWS or MS Azure)\n\nThese skills are critical for designing and implementing software components, participating in code reviews, and assisting customers in leveraging the services offered by Microsoft Fabri

In [8]:
tp_skls = top_skills['top_skills'].to_list()

In [10]:
tp_skls[0].split('#')

['Proficient coding skills in Java or Scala or Python',
 'Knowledge of SQL',
 'Familiarity with distributed systems like Spark',
 'Git',
 'Maven',
 'Azure DevOps',
 'Data Engineering',
 'Data Science',
 'Data Integration',
 'Experience in Cloud Developer toolchains (AWS or MS Azure)']

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', cache_folder="C:\\Users\\aanur\\projects\\jobs-data-mining\\mount\\all-MiniLM-L6-v2")

In [50]:
embs = model.encode(tp_skls[0].split('#'))

In [52]:
embs.shape

(384,)

In [86]:
import numpy as np

embs = np.array([])
for sk in tp_skls[0].split('#'):
    if embs.shape[0] == 0:
        embs = model.encode(sk).reshape(1, -1)
    else:
        embs = np.append(embs, model.encode(sk).reshape(1, -1), axis=0)
    print(embs.shape)

(1, 384)
(2, 384)
(3, 384)
(4, 384)
(5, 384)
(6, 384)
(7, 384)
(8, 384)
(9, 384)
(10, 384)


In [19]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [22]:
top_skills.iloc[0].to_dict()

{'id': 1,
 'entry_date': Timestamp('2024-09-01 06:38:10'),
 'job_id': '%E2%80%8B%E2%80%8Bsoftware-engineer%E2%80%8B-at-microsoft-3974756992',
 'top_skills': 'Proficient coding skills in Java or Scala or Python#Knowledge of SQL#Familiarity with distributed systems like Spark#Git#Maven#Azure DevOps#Data Engineering#Data Science#Data Integration#Experience in Cloud Developer toolchains (AWS or MS Azure)',
 'number_of_skills': 10,
 'exact_llm_output': 'Based on the job description, here are the top skills required for the role:\n\n1. Proficient coding skills in Java or Scala or Python\n2. Knowledge of SQL\n3. Familiarity with distributed systems like Spark\n4. Git\n5. Maven\n6. Azure DevOps\n7. Data Engineering\n8. Data Science\n9. Data Integration\n10. Experience in Cloud Developer toolchains (AWS or MS Azure)\n\nThese skills are critical for designing and implementing software components, participating in code reviews, and assisting customers in leveraging the services offered by Microso

In [43]:
from qdrant_client.models import PointStruct
import uuid

job = top_skills.iloc[0].to_dict()
payload_base = {'job_id': job['job_id']}
points = [
        PointStruct(id=uuid.uuid4().__str__(), vector=model.encode(skill), payload={**payload_base, 'skill': skill, 'rank': rank+1})
        for rank, skill in enumerate(tp_skls[0].split('#'))
    ]

operation_info = client.upsert(
    collection_name="top_skills",
    wait=True,
    points=points,
)

In [49]:
operation_info.__dict__['status'].value

'completed'